In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
import tensorflow.keras.backend as K

# Performance metrics: Dice score coefficient, IOU, recall, sensitivity
def perf_metrics(y_true, y_pred):
    #print(np.max(y_true), np.min(y_true))
    #print(np.max(y_pred), np.min(y_pred))
    y_true = y_true/255.
    y_pred = y_pred/255.
    
    y_pred_pos = np.round(np.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos
    y_pos = np.round(np.clip(y_true, 0, 1)) # ground truth
    y_neg = 1 - y_pos
    tp = np.sum(y_pos * y_pred_pos)
    tn = np.sum(y_neg * y_pred_neg)
    fp = np.sum(y_neg * y_pred_pos)
    fn = np.sum(y_pos * y_pred_neg)
    tpr = (tp + K.epsilon()) / (tp + fn + K.epsilon()) #recall
    tnr = (tn + K.epsilon()) / (tn + fp + K.epsilon())
    prec = (tp + K.epsilon()) / (tp + fp + K.epsilon()) #precision
    iou = (tp + K.epsilon()) / (tp + fn + fp + K.epsilon()) #intersection over union
    dsc = (2*tp + K.epsilon()) / (2*tp + fn + fp + K.epsilon()) #dice score
    return [dsc, iou, tpr, prec]

## K-Means

In [5]:
# Import the required libraries.
import cv2
import glob
import warnings
import numpy as np
# import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from time import time

warnings.filterwarnings("ignore")


# K-Means Image Segmentation.
class KMeansSegment(object):
    # Skull strip the brain scans.
    def skullStrip(self, images):
        gray_imgs = []
        brains = []
        for i in range(len(images)):
            gray_img = cv2.cvtColor(images[i], cv2.COLOR_BGR2GRAY)
            gray_imgs.append(gray_img)
            ret, thresh = cv2.threshold(gray_imgs[i], 0, 255, cv2.THRESH_OTSU)
            colormask = np.zeros(images[i].shape, dtype=np.uint8)
            colormask[thresh != 0] = np.array((0, 0, 255))
            ret, markers = cv2.connectedComponents(thresh)
            marker_area = [
                    np.sum(markers == m)
                    for m in range(np.max(markers)) if m != 0
            ]
            largest_component = np.argmax(marker_area)+1
            brain_mask = markers == largest_component
            brain_out = images[i].copy()
            brain_out[brain_mask == False] = (0, 0, 0)
            brains.append(brain_out)
        return brains

    # Process the skull stripped images.
    def readSkulls(self, brains):
        skull_imgs = []
        for i in range(len(brains)):
            img = cv2.resize(brains[i], (256, 256))
            rgb_img = img.reshape((img.shape[0] * img.shape[1], 3))
            skull_imgs.append(rgb_img)
        return skull_imgs

    # Compute the area of the objects in a binary image.
    def bwarea(self, img):
        row = img.shape[0]
        col = img.shape[1]
        total = 0.0
        for r in range(row-1):
            for c in range(col-1):
                sub_total = img[r:r+2, c:c+2].mean()
                if sub_total == 255:
                    total += 1
                elif sub_total == (255.0/3.0):
                    total += (7.0/8.0)
                elif sub_total == (255.0/4.0):
                    total += 0.25
                elif sub_total == 0:
                    total += 0
                else:
                    r1c1 = img[r, c]
                    r1c2 = img[r, c+1]
                    r2c1 = img[r+1, c]
                    r2c2 = img[r+1, c+1]

                    if (((r1c1 == r2c2) & (r1c2 == r2c1)) & (r1c1 != r2c1)):
                        total += 0.75
                    else:
                        total += 0.5
        return total

    # Change the cluster color.
    def change_color_kmeans(self, predict_img, clusters):
        img = []
        for val in predict_img:
            img.append(clusters[val])
        return img

    # Implementing K-Means algorithm.
    def process(self, list_img, clusters):
        self.seg_array = []
        for rgb_img in list_img[:]:
            img = np.reshape(rgb_img, (256, 256, 3)).astype(np.uint8)
            shape = np.shape(img)

            clt = KMeans(n_clusters=clusters)
            clt.fit(rgb_img)
            predict_img = clt.predict(rgb_img)
            new_img = self.change_color_kmeans(
                    predict_img,
                    clt.cluster_centers_
                    )
            kmeans_img = np.reshape(new_img, shape).astype(np.uint8)
            new_time = time()

            self.ret, self.seg_img = cv2.threshold(
                    kmeans_img,
                    np.max(kmeans_img) - 1,
                    255,
                    cv2.THRESH_BINARY
            )
            print("Time:", time() - new_time, 'seconds')
            print("BWAREA:", self.bwarea(self.seg_img[:, :, 1]))

            self.seg_array.append(self.seg_img)

            # Un-comment if you want to display the images via matplotlib.
            """
            plt.subplot(131)
            plt.gca().set_title("Original"), plt.imshow(img)
            plt.xticks([]), plt.yticks([])

            plt.subplot(132)
            plt.gca().set_title("K-Means"), plt.imshow(kmeans_img)
            plt.xticks([]), plt.yticks([])

            plt.subplot(133)
            plt.gca().set_title("Segmented"), plt.imshow(self.seg_img)
            plt.xticks([]), plt.yticks([])
            plt.show()
            """

    def metrics(self, masks):
      dsc_mean = []
      iou_mean = []
      tpr_mean = []
      prec_mean = []
      print(self.seg_array[0].shape)
      for i in range(len(masks)):
        dsc, iou, tpr, prec = perf_metrics(masks[i], self.seg_array[i])
        dsc_mean.append(dsc)
        iou_mean.append(iou)
        tpr_mean.append(tpr)
        prec_mean.append(prec)

      return sum(dsc_mean)/len(dsc_mean), sum(iou_mean)/len(iou_mean), sum(tpr_mean)/len(tpr_mean), sum(prec_mean)/len(prec_mean)  


    # Write the image to chosen directory.
    def writeFile(self, OUTDIR):
        for i in range(len(self.seg_array)):
            cv2.imwrite(OUTDIR + "Result_" + str(i) + ".png", self.seg_array[i])

In [ ]:
# Main method.
import natsort

PATH = "/content/drive/MyDrive/UNET-Data/Coronal/"
MASK_PATH = "/content/drive/MyDrive/UNET-Data/Coronal-Mask/"

img_path = natsort.natsorted(glob.glob(PATH + "*.png"))
mask_path = natsort.natsorted(glob.glob(MASK_PATH + "*.png"))
print(list(zip(img_path, mask_path)))

images = [cv2.resize(cv2.imread(file), (256, 256)) for file in img_path]
masks =  [cv2.resize(cv2.imread(file), (256,256)) for file in mask_path]

means = KMeansSegment()
skulls = means.skullStrip(images)
proc_skulls = means.readSkulls(skulls)
means.process(proc_skulls, 4)
dsc, iou, tpr, prec = means.metrics(masks)
print(dsc, iou, tpr, prec)

[('/content/drive/MyDrive/UNET-Data/Coronal/Image_4_Coronal_114.png', '/content/drive/MyDrive/UNET-Data/Coronal-Mask/Mask_4_Coronal_114.png'), ('/content/drive/MyDrive/UNET-Data/Coronal/Image_4_Coronal_115.png', '/content/drive/MyDrive/UNET-Data/Coronal-Mask/Mask_4_Coronal_115.png'), ('/content/drive/MyDrive/UNET-Data/Coronal/Image_4_Coronal_116.png', '/content/drive/MyDrive/UNET-Data/Coronal-Mask/Mask_4_Coronal_116.png'), ('/content/drive/MyDrive/UNET-Data/Coronal/Image_4_Coronal_117.png', '/content/drive/MyDrive/UNET-Data/Coronal-Mask/Mask_4_Coronal_117.png'), ('/content/drive/MyDrive/UNET-Data/Coronal/Image_4_Coronal_118.png', '/content/drive/MyDrive/UNET-Data/Coronal-Mask/Mask_4_Coronal_118.png'), ('/content/drive/MyDrive/UNET-Data/Coronal/Image_4_Coronal_119.png', '/content/drive/MyDrive/UNET-Data/Coronal-Mask/Mask_4_Coronal_119.png'), ('/content/drive/MyDrive/UNET-Data/Coronal/Image_4_Coronal_120.png', '/content/drive/MyDrive/UNET-Data/Coronal-Mask/Mask_4_Coronal_120.png'), ('/co

In [ ]:
# Main method.
import natsort

PATH = "/content/drive/MyDrive/UNET-Data/Sagittal/"
MASK_PATH = "/content/drive/MyDrive/UNET-Data/Sagittal-Mask/"

img_path = natsort.natsorted(glob.glob(PATH + "*.png"))
mask_path = natsort.natsorted(glob.glob(MASK_PATH + "*.png"))
print(list(zip(img_path, mask_path)))

images = [cv2.resize(cv2.imread(file), (256, 256)) for file in img_path]
masks =  [cv2.resize(cv2.imread(file), (256,256)) for file in mask_path]

means = KMeansSegment()
skulls = means.skullStrip(images)
proc_skulls = means.readSkulls(skulls)
means.process(proc_skulls, 4)
dsc, iou, tpr, prec = means.metrics(masks)
print(dsc, iou, tpr, prec)

[('/content/drive/MyDrive/UNET-Data/Sagittal/Image_2_Sagittal_156.png', '/content/drive/MyDrive/UNET-Data/Sagittal-Mask/Mask_2_Sagittal_156.png'), ('/content/drive/MyDrive/UNET-Data/Sagittal/Image_2_Sagittal_157.png', '/content/drive/MyDrive/UNET-Data/Sagittal-Mask/Mask_2_Sagittal_157.png'), ('/content/drive/MyDrive/UNET-Data/Sagittal/Image_2_Sagittal_158.png', '/content/drive/MyDrive/UNET-Data/Sagittal-Mask/Mask_2_Sagittal_158.png'), ('/content/drive/MyDrive/UNET-Data/Sagittal/Image_2_Sagittal_159.png', '/content/drive/MyDrive/UNET-Data/Sagittal-Mask/Mask_2_Sagittal_159.png'), ('/content/drive/MyDrive/UNET-Data/Sagittal/Image_2_Sagittal_160.png', '/content/drive/MyDrive/UNET-Data/Sagittal-Mask/Mask_2_Sagittal_160.png'), ('/content/drive/MyDrive/UNET-Data/Sagittal/Image_2_Sagittal_161.png', '/content/drive/MyDrive/UNET-Data/Sagittal-Mask/Mask_2_Sagittal_161.png'), ('/content/drive/MyDrive/UNET-Data/Sagittal/Image_2_Sagittal_162.png', '/content/drive/MyDrive/UNET-Data/Sagittal-Mask/Mask

In [ ]:
# Main method.
import natsort

PATH = "/content/drive/MyDrive/UNET-Data/Transversal/"
MASK_PATH = "/content/drive/MyDrive/UNET-Data/Transversal-Mask/"

img_path = natsort.natsorted(glob.glob(PATH + "*.png"))
mask_path = natsort.natsorted(glob.glob(MASK_PATH + "*.png"))
print(list(zip(img_path, mask_path)))

images = [cv2.resize(cv2.imread(file), (256, 256)) for file in img_path]
masks =  [cv2.resize(cv2.imread(file), (256,256)) for file in mask_path]

means = KMeansSegment()
skulls = means.skullStrip(images)
proc_skulls = means.readSkulls(skulls)
means.process(proc_skulls, 4)
dsc, iou, tpr, prec = means.metrics(masks)
print(dsc, iou, tpr, prec)

[('/content/drive/MyDrive/UNET-Data/Transversal/Image_1_Transversal_63.png', '/content/drive/MyDrive/UNET-Data/Transversal-Mask/Mask_1_Transversal_63.png'), ('/content/drive/MyDrive/UNET-Data/Transversal/Image_1_Transversal_64.png', '/content/drive/MyDrive/UNET-Data/Transversal-Mask/Mask_1_Transversal_64.png'), ('/content/drive/MyDrive/UNET-Data/Transversal/Image_1_Transversal_65.png', '/content/drive/MyDrive/UNET-Data/Transversal-Mask/Mask_1_Transversal_65.png'), ('/content/drive/MyDrive/UNET-Data/Transversal/Image_1_Transversal_66.png', '/content/drive/MyDrive/UNET-Data/Transversal-Mask/Mask_1_Transversal_66.png'), ('/content/drive/MyDrive/UNET-Data/Transversal/Image_1_Transversal_67.png', '/content/drive/MyDrive/UNET-Data/Transversal-Mask/Mask_1_Transversal_67.png'), ('/content/drive/MyDrive/UNET-Data/Transversal/Image_1_Transversal_68.png', '/content/drive/MyDrive/UNET-Data/Transversal-Mask/Mask_1_Transversal_68.png'), ('/content/drive/MyDrive/UNET-Data/Transversal/Image_1_Transver

## Fuzzy C Means

In [3]:
!pip install scikit-fuzzy

     |████████████████████████████████| 993 kB 12.1 MB/s 
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-py3-none-any.whl size=894089 sha256=fd349f51e7f684ea326bcef59480fd551377036eb9701564bba80a432d434069
  Stored in directory: /root/.cache/pip/wheels/d5/74/fc/38588a3d2e3f34f74588e6daa3aa5b0a322bd6f9420a707131
Successfully built scikit-fuzzy


In [7]:
# Import the required libraries.
from __future__ import division, print_function

import os
import cv2
import warnings
import numpy as np
import skfuzzy as fuzz

from sklearn.cluster import KMeans
from time import time

warnings.filterwarnings("ignore")


# Class creation
class FuzzyC(object):
    # Read in the images.
    def readImage(self, folder):
        list_images = natsort.natsorted(os.listdir(folder))
        list_img = []
        for i in list_images:
            path = folder+i
            img = cv2.imread(path)
            img = cv2.resize(img, (256, 256))
            img = cv2.medianBlur(img, 5)
            rgb_img = img.reshape((img.shape[0] * img.shape[1], 3))
            list_img.append(rgb_img)

        return list_img

    # Compute the area of the objects in a binary image.
    def bwarea(self, img):
        row = img.shape[0]
        col = img.shape[1]
        total = 0.0
        for r in range(row-1):
            for c in range(col-1):
                sub_total = img[r:r+2, c:c+2].mean()
                if sub_total == 255:
                    total += 1
                elif sub_total == (255.0/3.0):
                    total += (7.0/8.0)
                elif sub_total == (255.0/4.0):
                    total += 0.25
                elif sub_total == 0:
                    total += 0
                else:
                    r1c1 = img[r, c]
                    r1c2 = img[r, c+1]
                    r2c1 = img[r+1, c]
                    r2c2 = img[r+1, c+1]

                    if (((r1c1 == r2c2) & (r1c2 == r2c1)) & (r1c1 != r2c1)):
                        total += 0.75
                    else:
                        total += 0.5
        return total

    def centroid_histogram(self, clt):
        numLabels = np.arange(0, len(np.unique(clt.labels_)) + 1)
        (hist, _) = np.histogram(clt.labels_, bins=numLabels)

        hist = hist.astype("float")
        hist /= hist.sum()

        return hist

    def plot_colors(self, hist, centroids):
        bar = np.zeros((50, 300, 3), dtype="uint8")
        startX = 0
        for (percent, color) in zip(hist, centroids):
            endX = startX + (percent * 300)
            cv2.rectangle(
                    bar,
                    (int(startX), 0),
                    (int(endX), 50),
                    color.astype("uint8").tolist(),
                    -1
            )
            startX = endX

        return bar

    def change_color_kmeans(self, predict_img, clusters):
        img = []
        for val in predict_img:
            img.append(clusters[val])
        return img

    def change_color_fuzzycmeans(self, cluster_membership, clusters):
        img = []
        for pix in cluster_membership.T:
            img.append(clusters[np.argmax(pix)])
        return img

    def process(self, list_img, clusters):
        self.seg_array = []
        for rgb_img in list_img[:]:
            img = np.reshape(rgb_img, (256, 256, 3)).astype(np.uint8)
            shape = np.shape(img)

            clt = KMeans(n_clusters=clusters)
            clt.fit(rgb_img)
            predict_img = clt.predict(rgb_img)
            new_img = self.change_color_kmeans(
                    predict_img,
                    clt.cluster_centers_
            )
            # kmeans_img = np.reshape(new_img, shape).astype(np.uint8)
            # hist = self.centroid_histogram(clt)
            # bar = self.plot_colors(hist, clt.cluster_centers_)
            new_time = time()

            cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(
                    rgb_img.T,
                    clusters,
                    2,
                    error=0.005,
                    maxiter=1000,
                    init=None,
                    seed=42
            )

            new_img = self.change_color_fuzzycmeans(u, cntr)
            fuzzy_img = np.reshape(new_img, shape).astype(np.uint8)
            self.ret, self.seg_img = cv2.threshold(
                    fuzzy_img,
                    np.max(fuzzy_img) - 1,
                    255,
                    cv2.THRESH_BINARY
            )
            print(time() - new_time, 'seconds')
            print(self.bwarea(self.seg_img[:, :, 1]))

            self.seg_array.append(self.seg_img)

            """
            plt.subplot(131)
            plt.gca().set_title("Original"), plt.imshow(img)
            plt.xticks([]), plt.yticks([])

            plt.subplot(132)
            plt.gca().set_title("Fuzzy"), plt.imshow(fuzzy_img)
            plt.xticks([]), plt.yticks([])

            plt.subplot(133)
            plt.gca().set_title("Segmented"), plt.imshow(self.seg_img)
            plt.xticks([]), plt.yticks([])
            plt.show()
            """

    def metrics(self, masks):
      dsc_mean = []
      iou_mean = []
      tpr_mean = []
      prec_mean = []
      print(self.seg_array[0].shape)
      for i in range(len(masks)):
        dsc, iou, tpr, prec = perf_metrics(masks[i], self.seg_array[i])
        dsc_mean.append(dsc)
        iou_mean.append(iou)
        tpr_mean.append(tpr)
        prec_mean.append(prec)

      return sum(dsc_mean)/len(dsc_mean), sum(iou_mean)/len(iou_mean), sum(tpr_mean)/len(tpr_mean), sum(prec_mean)/len(prec_mean)  

    def writeFile(self, OUTDIR):
        for i in range(len(self.seg_array)):
            cv2.imwrite(OUTDIR + "Result_" + str(i) + ".png", self.seg_array[i])

In [ ]:
PATH = "/content/drive/MyDrive/UNET-Data/Coronal/"
MASK_PATH = "/content/drive/MyDrive/UNET-Data/Coronal-Mask/"

mask_path = natsort.natsorted(glob.glob(MASK_PATH + "*.png"))
print(list(zip(img_path, mask_path)))
masks =  [cv2.resize(cv2.imread(file), (256,256)) for file in mask_path]

fuzzy = FuzzyC()
list_img = fuzzy.readImage(PATH)
fuzzy.process(list_img, 6)
dsc, iou, tpr, prec = fuzzy.metrics(masks)
print(dsc, iou, tpr, prec )

[('/content/drive/MyDrive/UNET-Data/Transversal/Image_1_Transversal_63.png', '/content/drive/MyDrive/UNET-Data/Coronal-Mask/Mask_4_Coronal_114.png'), ('/content/drive/MyDrive/UNET-Data/Transversal/Image_1_Transversal_64.png', '/content/drive/MyDrive/UNET-Data/Coronal-Mask/Mask_4_Coronal_115.png'), ('/content/drive/MyDrive/UNET-Data/Transversal/Image_1_Transversal_65.png', '/content/drive/MyDrive/UNET-Data/Coronal-Mask/Mask_4_Coronal_116.png'), ('/content/drive/MyDrive/UNET-Data/Transversal/Image_1_Transversal_66.png', '/content/drive/MyDrive/UNET-Data/Coronal-Mask/Mask_4_Coronal_117.png'), ('/content/drive/MyDrive/UNET-Data/Transversal/Image_1_Transversal_67.png', '/content/drive/MyDrive/UNET-Data/Coronal-Mask/Mask_4_Coronal_118.png'), ('/content/drive/MyDrive/UNET-Data/Transversal/Image_1_Transversal_68.png', '/content/drive/MyDrive/UNET-Data/Coronal-Mask/Mask_4_Coronal_119.png'), ('/content/drive/MyDrive/UNET-Data/Transversal/Image_1_Transversal_69.png', '/content/drive/MyDrive/UNET-

In [ ]:
PATH = "/content/drive/MyDrive/UNET-Data/Sagittal/"
MASK_PATH = "/content/drive/MyDrive/UNET-Data/Sagittal-Mask/"

mask_path = natsort.natsorted(glob.glob(MASK_PATH + "*.png"))
print(list(zip(img_path, mask_path)))
masks =  [cv2.resize(cv2.imread(file), (256,256)) for file in mask_path]

fuzzy = FuzzyC()
list_img = fuzzy.readImage(PATH)
fuzzy.process(list_img, 6)
dsc, iou, tpr, prec = fuzzy.metrics(masks)
print(dsc, iou, tpr, prec )

[('/content/drive/MyDrive/UNET-Data/Transversal/Image_1_Transversal_63.png', '/content/drive/MyDrive/UNET-Data/Sagittal-Mask/Mask_2_Sagittal_156.png'), ('/content/drive/MyDrive/UNET-Data/Transversal/Image_1_Transversal_64.png', '/content/drive/MyDrive/UNET-Data/Sagittal-Mask/Mask_2_Sagittal_157.png'), ('/content/drive/MyDrive/UNET-Data/Transversal/Image_1_Transversal_65.png', '/content/drive/MyDrive/UNET-Data/Sagittal-Mask/Mask_2_Sagittal_158.png'), ('/content/drive/MyDrive/UNET-Data/Transversal/Image_1_Transversal_66.png', '/content/drive/MyDrive/UNET-Data/Sagittal-Mask/Mask_2_Sagittal_159.png'), ('/content/drive/MyDrive/UNET-Data/Transversal/Image_1_Transversal_67.png', '/content/drive/MyDrive/UNET-Data/Sagittal-Mask/Mask_2_Sagittal_160.png'), ('/content/drive/MyDrive/UNET-Data/Transversal/Image_1_Transversal_68.png', '/content/drive/MyDrive/UNET-Data/Sagittal-Mask/Mask_2_Sagittal_161.png'), ('/content/drive/MyDrive/UNET-Data/Transversal/Image_1_Transversal_69.png', '/content/drive/M

In [ ]:
PATH = "/content/drive/MyDrive/UNET-Data/Transversal/"
MASK_PATH = "/content/drive/MyDrive/UNET-Data/Transversal-Mask/"

mask_path = natsort.natsorted(glob.glob(MASK_PATH + "*.png"))
print(list(zip(img_path, mask_path)))
masks =  [cv2.resize(cv2.imread(file), (256,256)) for file in mask_path]

fuzzy = FuzzyC()
list_img = fuzzy.readImage(PATH)
fuzzy.process(list_img, 6)
dsc, iou, tpr, prec = fuzzy.metrics(masks)
print(dsc, iou, tpr, prec )

[('/content/drive/MyDrive/UNET-Data/Transversal/Image_1_Transversal_63.png', '/content/drive/MyDrive/UNET-Data/Transversal-Mask/Mask_1_Transversal_63.png'), ('/content/drive/MyDrive/UNET-Data/Transversal/Image_1_Transversal_64.png', '/content/drive/MyDrive/UNET-Data/Transversal-Mask/Mask_1_Transversal_64.png'), ('/content/drive/MyDrive/UNET-Data/Transversal/Image_1_Transversal_65.png', '/content/drive/MyDrive/UNET-Data/Transversal-Mask/Mask_1_Transversal_65.png'), ('/content/drive/MyDrive/UNET-Data/Transversal/Image_1_Transversal_66.png', '/content/drive/MyDrive/UNET-Data/Transversal-Mask/Mask_1_Transversal_66.png'), ('/content/drive/MyDrive/UNET-Data/Transversal/Image_1_Transversal_67.png', '/content/drive/MyDrive/UNET-Data/Transversal-Mask/Mask_1_Transversal_67.png'), ('/content/drive/MyDrive/UNET-Data/Transversal/Image_1_Transversal_68.png', '/content/drive/MyDrive/UNET-Data/Transversal-Mask/Mask_1_Transversal_68.png'), ('/content/drive/MyDrive/UNET-Data/Transversal/Image_1_Transver